In [0]:
!pip install weighted_levenshtein
!pip install sklearn-pycrfsuite

     |████████████████████████████████| 491kB 13.8MB/s 
  Created wheel for sklearn-pycrfsuite: filename=sklearn_pycrfsuite-0.4.0-py2.py3-none-any.whl size=11002 sha256=9ca1ac42c6126d881b0e548c7348eb0007c11140ff39b8d562a1e6da18630999
  Stored in directory: /root/.cache/pip/wheels/af/78/bd/189a8a1d7a6031ce1e1f8e81a5a8bdb0d3fba7e88c53f96107
  Created wheel for python-crfsuite-extension: filename=python_crfsuite_extension-0.9.7-cp36-cp36m-linux_x86_64.whl size=779517 sha256=406d86a2eb466e34b0c65035bcbe096dc920670112f4add3ab25cb7cf72b6253
  Stored in directory: /root/.cache/pip/wheels/6d/ee/fa/10e4ae5dd9e71190ce1318fa7e2b82314bc880da1ec34ee567
Successfully built sklearn-pycrfsuite python-crfsuite-extension


In [0]:
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/My Drive/ner/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from weighted_levenshtein import lev, osa, dam_lev
from string import ascii_lowercase
import numpy as np
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

In [0]:
# baca file train dan test
train_lines = []
test_lines = []
with open(folder+'dataTrain.txt', 'r') as f:
    train_lines = f.readlines()
with open(folder+'kalimat_POSTag.txt', 'r') as f:
    test_lines = f.readlines()

In [0]:
#baca file list/gazeteer
per_list = []
flist = open(folder+"per_list.txt", "r")
for p in flist:
    per_list.append(p[:-1])

loc_list = []
llist = open(folder+"loc_list.txt", "r")
for l in llist:
    loc_list.append(l[:-1])

org_list = []
olist = open(folder+"org_list.txt", "r")
for o in olist:
    org_list.append(o[:-1])

In [0]:
train_sents = []
test_sents = []
sent = []
counter = 0
for line in train_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1], line_part[2])
        #print(t)
        sent.append(t)
    else:
        #print("train sent = ")
        #print(sent)
        train_sents.append(sent)
        sent = []
        counter = counter+1
counter = 0
sent = []
for line in test_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1])
        #print(t)
        sent.append(t)
    else:
        #print("test sent = ")
        #print(sent)
        test_sents.append(sent)
        sent = []
        counter = counter+1
test_sents.pop()

[]

In [0]:
#fungsi list lookup with edit distance
alfha = 0.4
threshold = 20

insert_costs = np.full(128, 100, dtype=np.float64)
insert_costs[ord('-')] = 10
insert_costs[ord(' ')] = 10

delete_costs = np.full(128, 100, dtype=np.float64)
delete_costs[ord('-')] = 10
delete_costs[ord(' ')] = 10

substitute_costs = np.full((128,128), 50, dtype=np.float64)
for c in ascii_lowercase:
	substitute_costs[ord(c), ord(c.capitalize())] = 10
	substitute_costs[ord(c), ord(c)] = 0
	substitute_costs[ord(c.capitalize()), ord(c)] = 10
	substitute_costs[ord(c.capitalize()), ord(c.capitalize())] = 0
substitute_costs[ord('-'), ord(' ')] = 10
substitute_costs[ord(' '), ord('-')] = 10
for i in range(10):
	for j in range(10):
		if i == j:
			substitute_costs[ord(str(i)), ord(str(j))] = 0
			substitute_costs[ord(str(j)), ord(str(i))] = 0
		else:
			substitute_costs[ord(str(i)), ord(str(j))] = 10
			substitute_costs[ord(str(j)), ord(str(i))] = 10

def edit_distance_normalized_cost(word, target):
	cost = lev(word, target, insert_costs=insert_costs, delete_costs=delete_costs, substitute_costs=substitute_costs)
	return (cost + alfha) / len(target)

def check_under_threshold(cost):
	if cost <= threshold:
		return True
	else:
		return False

def check_list(sentence, pos, list):
  words = sentence.split()
  candidate = []
  candidate.append(words[pos])
  if pos >= 0:
    if pos < (len(words) - 1):
      candidate.append(words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  if (pos - 1) >= 0:
    candidate.append(words[pos - 1] + " " + words[pos])
    if pos < (len(words) - 1):
      candidate.append(words[pos - 1] + " " + words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos - 1] + " " + words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  if (pos - 2) >= 0:
    candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos])
    if pos < (len(words) - 1):
      candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos] + " " + words[pos + 1])
    if pos < (len(words) - 2):
      candidate.append(words[pos - 2] + " " + words[pos - 1] + " " + words[pos] + " " + words[pos + 1] + " " + words[pos + 2])
  exist = 'False'
  for l in list:
    for c in candidate:
      if check_under_threshold(edit_distance_normalized_cost(c,l)) :
        exist = 'True'
        break
  return exist
print(check_list('Universitas Gadjah Mada',0,org_list))

True


In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    sentence = ''
    for w in sent:
      sentence += w[0] + " "
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
        'inPerList=' + check_list(sentence,i,per_list), #check di list person
        'inLocList=' + check_list(sentence,i,loc_list), #check di list location
        'inOrgList=' + check_list(sentence,i,org_list) #check di list organisation
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent] 

In [0]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
#y_test = [sent2labels(s) for s in test_sents]

In [0]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train('sample.crfsuite')

tagger = pycrfsuite.Tagger()
tagger.open('sample.crfsuite')
file = open(folder+'crf_output.txt','w') 
#example_sent = test_sents[0]
#print(' '.join(sent2tokens(example_sent)), end='\n\n')
for zz in test_sents:
  pred = tagger.tag(sent2features(zz))
  for ww in range(len(zz)):
    #print(zz[ww][0],pred[ww])
    file.write(zz[ww][0])
    file.write('\t')
    file.write(pred[ww])
    file.write('\n')
  #print('\n')
  file.write('\n')
print('finished')
file.close()
#print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
#print("Correct:  ", ' '.join(sent2labels(example_sent)))

finished
